In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers


In [10]:
data_dir = "../../../../Dataset/Soil types"
soils = os.listdir(data_dir)

In [11]:
soils

['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil']

In [13]:
# Number of images for each disease
nums = {}
for soil in soils:
    nums[soil] = len(os.listdir(data_dir + '/' + soil))

# converting the nums dictionary to pandas dataframe passing index as plant name and number of images as column

img_per_class = pd.DataFrame(nums.values(), index=nums.keys(), columns=["no. of images"])
img_per_class

,no. of images
Black Soil,37
Cinder Soil,30
Laterite Soil,30
Peat Soil,30
Yellow Soil,29


In [14]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_set = datagen.flow_from_directory(data_dir,
                                             target_size=(256, 256),
                                             batch_size=10,
                                             class_mode='categorical',
                                             subset='training')
test_set = datagen.flow_from_directory(data_dir,
                                             target_size=(256, 256),
                                             batch_size=10,
                                             class_mode='categorical',
                                             subset='validation')


Found 126 images belonging to 5 classes.
Found 30 images belonging to 5 classes.


In [15]:
class_labels = list(train_set.class_indices.keys())
class_labels

['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil']

In [ ]:
# Model instantiation
model = Sequential()

# Convolutional layers
model.add(Conv2D(64, (3, 3), input_shape=(256, 256, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Flatten layer
model.add(Flatten())

# Hidden layers with regularization
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(len(train_set.class_indices), activation='softmax'))

# Model compilation
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 64)      0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 127, 127, 64)      256       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 128)       0         
 g2D)                                                            
                                                        

In [ ]:
# Model training
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
soil_model = model.fit(train_set, epochs=20, validation_data=test_set, callbacks=[early_stopping])

Epoch 1/20
14/14 [==============================] - 151s 11s/step - loss: 5.6864 - accuracy: 0.8321 - val_loss: 6.7471 - val_accuracy: 0.4839
Epoch 2/20
14/14 [==============================] - 137s 10s/step - loss: 5.6356 - accuracy: 0.8015 - val_loss: 6.1453 - val_accuracy: 0.5484
Epoch 3/20
14/14 [==============================] - 139s 10s/step - loss: 5.1586 - accuracy: 0.8702 - val_loss: 5.9696 - val_accuracy: 0.5806
Epoch 4/20
14/14 [==============================] - 153s 11s/step - loss: 5.2255 - accuracy: 0.8092 - val_loss: 6.2479 - val_accuracy: 0.5484
Epoch 5/20
14/14 [==============================] - 144s 10s/step - loss: 5.1018 - accuracy: 0.8092 - val_loss: 5.4395 - val_accuracy: 0.4839
Epoch 6/20
14/14 [==============================] - 136s 10s/step - loss: 4.7921 - accuracy: 0.8092 - val_loss: 5.4511 - val_accuracy: 0.4839
Epoch 7/20
14/14 [==============================] - 147s 11s/step - loss: 4.7496 - accuracy: 0.8092 - val_loss: 4.9442 - val_accuracy: 0.6452
Epoch 

In [ ]:
# Load and preprocess the new image
img_path = '/content/laterite-soil.jpeg'  
img = image.load_img(img_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the pixel values

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Map the predicted class index to the class label
class_labels = ['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil']
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)


1/1 [==============================] - 1s 752ms/step
Predicted class: Laterite Soil


In [ ]:
# Load and preprocess the new image
img_path = '/content/peat-soil.jpg'  
img = image.load_img(img_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the pixel values

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Map the predicted class index to the class label
class_labels = list(train_set.class_indices.keys())
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)

1/1 [==============================] - 0s 265ms/step
Predicted class: Peat Soil


In [ ]:
# Load and preprocess the new image
img_path = '/content/yellow-soil.jpg' 
img = image.load_img(img_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the pixel values

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Map the predicted class index to the class label
class_labels = list(train_set.class_indices.keys())
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)

1/1 [==============================] - 0s 320ms/step
Predicted class: Yellow Soil


In [ ]:
# Save the model to Google Drive
model.save('/content/drive/MyDrive/model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('../models/model.h5')

In [4]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the new image
img_path = './2.jfif'  
img = image.load_img(img_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the pixel values

# Make predictions
predictions = loaded_model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Map the predicted class index to the class label
class_labels = ['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil']
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)

1/1 [==============================] - 2s 2s/step
Predicted class: Yellow Soil
